In [47]:
from openai import OpenAI
# from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser


In [36]:
import os
from dotenv import load_dotenv

In [37]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Import Prompt

create new LLM, insert prompt and start chain

In [38]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()

In [ ]:
print(prompt_continuous_effects)

## RAG
### Loader

In [39]:
loader = TextLoader("./data/rules_shortened.txt")
rules_doc = loader.load()

### Splitter

In [40]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
print(rules_splits[0])

page_content='611. Continuous Effects\n\n611.1. A continuous effect modifies characteristics of objects, modifies control of objects, or affects players or the rules of the game, for a fixed or indefinite period.\n\n611.2. A continuous effect may be generated by the resolution of a spell or ability.\n\n611.2a A continuous effect generated by the resolution of a spell or ability lasts as long as stated by the spell or ability creating it (such as “until end of turn”). If no duration is stated, it lasts until the end of the game.' metadata={'source': './data/rules_shortened.txt'}


### Vector Store

In [53]:
rules_db = Chroma.from_documents(rules_splits, OpenAIEmbeddings())
retriever = rules_db.as_retriever()

- create new LLM
- load template

In [42]:
llm = OpenAI(openai_api_key=openai_api_key)


In [54]:
prompt_template = PromptTemplate.from_template(prompt_continuous_effects)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- Give Cardnames in prompt
- LCEL for prompt variables

In [57]:
#TODO: cards can't passed in chains as strings. Maybe using json data from mtgjson OR from scryfall via http get request

card1 = "Blood Moon"
card2 = "Urborg, Tomb of Yawgmoth"
chain = ({"card1": card1, "card2": card2, "rules_db": retriever} | prompt_template | llm | StrOutputParser())
chain.invoke(f"What happens if player a plays {card1} and player b plays {card2}?")

KeyError: "Input to PromptTemplate is missing variables {'card1', 'card2'}.  Expected: ['card1', 'card2', 'rules_db'] Received: ['rules_db']"

In [10]:
prompt = prompt_template.invoke({"card1": "Blood Moon", "card2": "Tomb of Yawgmoth", "rules_db": rules_db})
llm.invoke(prompt)

'\n\n1. Blood Moon: Nonbasic lands are Mountains.\nTomb of Yawgmoth: Each swamp is a 1/1 black and Phyrexian creature with "Sacrifice this creature: Add {B}."\n\n2. According to the official Magic: the Gathering rules, continuous effects are those that continuously change the characteristics of objects, spells, or players. They are often created by the resolution of spells or abilities, and last until they are overwritten by a new continuous effect or the game ends. These effects can be applied in layers, with the most recently created effect being applied last.\n\n2a. In paragraph 613: Interaction of continuous effects, it is stated that if multiple continuous effects are applied to the same object, they are applied in timestamp order, meaning the most recently created effect is applied last.\n\n3. In this case, both Blood Moon and Tomb of Yawgmoth have continuous effects that affect the same game object (lands). Based on the timestamp rule in 613.7, the most recently created effect w

## Case 2: Opalescence vs Humility

In [21]:
prompt_val = prompt_template.invoke({"card1": "Opalescence", "card2": "Humility"})

In [22]:
llm.invoke(prompt)

"\n\n1. Opalescence - Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its converted mana cost.\n\nHumility - All creatures lose all abilities and have base power and toughness 1/1.\n\n2. According to the official Magic: the Gathering rules, continuous effects are those that continuously change the characteristics of objects or players. These effects can come from various sources, such as spells, abilities, and game rules.\n\n2a. In paragraph 613, the rules explain how multiple continuous effects interact with each other. It states that if two or more effects modify the same characteristic of an object, the one with the most recent timestamp is the one that applies.\n\n3. In this case, the timestamp rule 613.7 would apply. This is because both Opalescence and Humility have continuous effects that modify the same characteristic of creatures - their power and toughness. However, since Opalescence's effect has

In [23]:
prompt = prompt_template.invoke({"card1": "Humility", "card2": "Opalescence"})

In [24]:
llm.invoke(prompt)

'\n\n1. Humility: All creatures lose all abilities and have base power and toughness 1/1.\nOpalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and toughness each equal to its converted mana cost.\n\n2. Rule 613: Interaction of Continuous Effects\n613.1. The continuous effects of an object are determined by the text of that object and may be modified by other continuous effects. \n613.2. Within layer 1, continuous effects are applied in a series of sublayers in the order described below. Within each sublayer, apply effects in timestamp order (see rule 613.7). Note that dependency may alter the order in which effects are applied within a sublayer. \n613.7. Within a sublayer or within an object, determine the timestamp first, then apply effects in timestamp order. Within each timestamp, apply effects in a series of sublayers in the order described above. \n613.8. Dependency may alter the order in which continuous effects are applied